# Follow-up Detection Production Pipeline

This notebook implements an efficient follow-up detection algorithm for radiology reports using GPT-OSS:20b.

**Architecture:**
- Parallel classification with GPT-OSS:20b using ThreadPoolExecutor (16 workers)
- Results written back to `latest_reports` with confidence scoring
- Errors logged to `followup_errors` table for debugging

**Performance:**
- Throughput: ~2.0 reports/second with 16 workers
- Estimated Runtime: ~5.8 days for 1M reports

**Columns Added to `latest_reports`:**
- `followup_detected` (boolean): True if follow-up recommended
- `followup_confidence` (string): "high" or "low"
- `followup_snippet` (string): Exact text indicating follow-up
- `followup_finding` (string): Finding type from LLM classification (e.g., "Pulmonary nodule")
- `followup_processed_at` (timestamp): Processing timestamp

**Error Handling:**
- Processing errors are logged to `followup_errors` table with report ID and error message
- Useful for debugging LLM issues, JSON parsing failures, or timeout errors

## Imports & Configuration

In [1]:
from pyspark.sql import SparkSession, functions as F, Window
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, TimestampType
from datetime import datetime
from tqdm.notebook import tqdm
import requests
import json
import os
import re
import time

# Configuration
OLLAMA_URL = os.getenv("OLLAMA_URL", "http://ollama-gpu1.ollama-dual:11434")
MODEL = "gpt-oss:20b"
S3_ACCESS_KEY = os.getenv("S3_ACCESS_KEY", "lake-writer")
S3_SECRET_KEY = os.getenv("S3_SECRET_KEY", "f8bae22e21dd400ba8b8b245b5b7c5ad62e4667ae201e411006caa55225ec2b4")

if not S3_SECRET_KEY:
    from getpass import getpass
    S3_SECRET_KEY = getpass("Enter S3 secret key: ")

# Threading configuration
TOTAL_WORKERS = 16  # Match OLLAMA_NUM_PARALLEL for optimal throughput

# Processing configuration
BATCH_SIZE = 1000

# Priority filtering - process reports with follow-up keywords first
FOLLOWUP_KEYWORDS = [
    r"follow.?up", r"repeat", r"recommend", r"suggest", r"interval",
    r"re-?evaluate", r"re-?assess", r"correlat", r"clinical", r"short.?term",
    r"further", r"additional", r"consider", r"refer", r"return",
    r"continue", r"monitor", r"surveillance", r"re-?exam", r"comparison"
]
PRIORITY_FILTER = f"lower(report_text) RLIKE '{('|'.join(FOLLOWUP_KEYWORDS))}'"

# Table names
SRC_TABLE = "default.reports"
DEST_TABLE = "default.latest_reports"
ERROR_TABLE = "default.followup_errors"

print(f"⚙️  Configuration:")
print(f"  Ollama URL: {OLLAMA_URL}")
print(f"  Model: {MODEL}")
print(f"  Total workers: {TOTAL_WORKERS}")
print(f"  Batch size: {BATCH_SIZE:,}")
print(f"  Priority filter: Keyword matches first")

⚙️  Configuration:
  Ollama URL: http://ollama-gpu1.ollama-dual:11434
  Model: gpt-oss:20b
  Total workers: 16
  Batch size: 1,000
  Priority filter: Keyword matches first


## Initialize Spark & Verify Connection

In [3]:
# Start Spark session with optimizations
spark = SparkSession.builder.appName("followup-detection") \
    .config("spark.hadoop.fs.s3a.access.key", S3_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", S3_SECRET_KEY) \
    .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore.hive:9083") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "800") \
    .enableHiveSupport().getOrCreate()

# Verify connection
try:
    spark.sql("SHOW DATABASES").show()
    print("✅ Spark connected to Delta Lake")
except Exception as e:
    print(f"❌ Error connecting to Delta Lake: {e}")
    raise

+---------+
|namespace|
+---------+
|  default|
+---------+

✅ Spark connected to Delta Lake


## Define Classification Functions

In [3]:
PROMPT_TEMPLATE = """Does this report recommend follow-up for THIS patient?

Follow-up = patient-specific imaging/evaluation for findings in THIS report

YES if finding-specific action:
- "Repeat CT in 6 months for nodule"
- "Clinical correlation recommended for lesion"
- Template applies: Patient HAS osteoporosis + template says "osteoporosis needs follow-up"

NO if:
- Normal: "unremarkable", "stable", "no acute findings"
- Template doesn't apply: Patient normal + template mentions other conditions
- Generic wellness: "all patients should take calcium"
- Follow-up exam completed, no future action requested

Check FINDINGS/IMPRESSION to see if templates apply to this patient.

Return ONLY valid JSON, nothing else:
{{"follow_up": true/false, "confidence": "high"/"low", "snippet": "text", "finding": "text"}}

If follow_up=false: snippet="" and finding=""
If follow_up=true: 
  - snippet=verbatim snippet of text from report recommending follow-up
  - finding=specific abnormality needing follow-up (e.g., "pulmonary nodule", "liver lesion")

Report:
{report_text}

JSON:"""

def classify_report(row):
    """Classify single report via Ollama API"""
    try:
        prompt = PROMPT_TEMPLATE.format(report_text=row.report_text)
        response = requests.post(
            f"{OLLAMA_URL}/api/generate",
            json={
                "model": MODEL,
                "prompt": prompt,
                "temperature": 0,
                "stream": False,
                "thinking": "low",
                "options": {
                    "num_predict": 1000,  # Prevent JSON truncation
                    "num_ctx": 4096
                }
            },
            timeout=120
        )
        
        if response.status_code != 200:
            raise Exception(f"HTTP {response.status_code}: {response.text}")
        
        resp_text = response.json()["response"].strip()

        # Check if response is empty
        if not resp_text:
            raise Exception("Empty response from model")

        # Extract JSON object, ignoring any text before or after
        
        # Remove code block fencing if present
        if resp_text.startswith("```json"):
            resp_text = resp_text[7:]  # Remove ```json
        if resp_text.startswith("```"):
            resp_text = resp_text[3:]  # Remove ```

        # Remove closing fence
        if "```" in resp_text:
            resp_text = resp_text.split("```")[0]
        
        resp_text = resp_text.strip()
        
        # Check again after cleanup
        if not resp_text:
            raise Exception("Empty response after cleanup")
        
        # Find first JSON object (handles text before/after)
        json_match = re.search(r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}', resp_text)
        if json_match:
            json_str = json_match.group()
        else:
            # Fallback: try to parse the whole thing
            json_str = resp_text.strip()

        # Final check before parsing
        if not json_str or not json_str.strip():
            raise Exception(f"No JSON found in response: {resp_text}")
        
        # Parse JSON
        obj = json.loads(json_str)
        
        if not isinstance(obj, dict):
            raise ValueError("Response is not a JSON object")
        
        # Extract and normalize values
        follow_up = bool(obj.get("follow_up", False))
        confidence = str(obj.get("confidence", "low")).lower()
        if confidence not in ["high", "low"]:
            confidence = "low"
        
        return {
            "message_control_id": row.message_control_id,
            "followup_detected": follow_up,
            "followup_confidence": confidence,
            "followup_snippet": str(obj.get("snippet", "")),
            "followup_finding": str(obj.get("finding", "")),
            "error": None
        }
        
    except Exception as e:
        return {
            "message_control_id": row.message_control_id,
            "followup_detected": None,
            "followup_confidence": None,
            "followup_snippet": None,
            "followup_finding": None,
            "error": str(e)
        }

    
print("✅ Classification function defined")

✅ Classification function defined


In [4]:
# Check processing progress
df_status = spark.table(DEST_TABLE)

total = df_status.count()
processed = df_status.filter(F.col("followup_processed_at").isNotNull()).count()
unprocessed = total - processed

print("📊 Processing Status:")
print(f"  Total reports: {total:,}")
print(f"  Processed: {processed:,} ({100*processed/total:.1f}%)")
print(f"  Unprocessed: {unprocessed:,}")

# Breakdown by follow-up status (for processed reports only)
if processed > 0:
  followup_stats = (df_status
                    .filter(F.col("followup_processed_at").isNotNull())
                    .groupBy("followup_detected", "followup_confidence")
                    .count()
                    .orderBy("followup_detected", "followup_confidence"))

  print("\n📈 Follow-up Detection Results:")
  followup_stats.show()

  # Summary metrics
  followup_count = df_status.filter(F.col("followup_detected") == True).count()
  print(f"\n✅ Follow-up rate: {100*followup_count/processed:.1f}% ({followup_count:,} / {processed:,})")

📊 Processing Status:
  Total reports: 12,875,092
  Processed: 358,415 (2.8%)
  Unprocessed: 12,516,677

📈 Follow-up Detection Results:
+-----------------+-------------------+------+
|followup_detected|followup_confidence| count|
+-----------------+-------------------+------+
|            false|               high|294719|
|            false|                low|  3821|
|             true|               high| 59810|
|             true|                low|    65|
+-----------------+-------------------+------+


✅ Follow-up rate: 16.7% (59,875 / 358,415)


## Process Reports with Checkpointing

In [ ]:
# Import parallel processing module
from parallel_processor import process_reports_in_batches

# Process all reports using simple ThreadPoolExecutor approach
# Priority filter processes keyword matches first
stats = process_reports_in_batches(
    spark=spark,
    classify_func=classify_report,
    dest_table=DEST_TABLE,
    error_table=ERROR_TABLE,
    ollama_url=OLLAMA_URL,
    model=MODEL,
    total_workers=TOTAL_WORKERS,
    batch_size=BATCH_SIZE,
    priority_filter=PRIORITY_FILTER  # Process keyword matches first
)

print("\n✅ Processing complete!")
print(f"\n📊 Final Statistics:")
print(f"  Total processed: {stats['total_processed']:,}")
print(f"  Total errors: {stats['total_errors']:,}")
print(f"  Total time: {stats['elapsed_time']/3600:.2f} hours")

📥 Loading model into memory...
✅ Model loaded into memory

🎯 Priority Filter Statistics:
  Priority reports (with keywords): 11,215,131 (89.6%)
  Non-priority reports: 1,301,546 (10.4%)

📊 Processing Summary:
  Total reports: 12,516,677
  Batch size: 1,000
  Batches: 12517
  Workers: 16
  Estimated time: 108.7 hours



Batches:   0%|          | 0/12517 [00:00<?, ?it/s]

  Batch 1 reports:   0%|          | 0/1000 [00:00<?, ?it/s]

  💾 Converting 1000 results to Spark DataFrame...
  📊 Counting successes and errors...
     ✓ 998 successful, 2 errors
  ⚠️  Writing 2 errors to default.followup_errors...
     ✓ Errors logged
  🔄 Merging 998 results into default.latest_reports...
     ✓ Merge complete
  📈 Calculating progress...
  📈 Batch 1/12517: 998 processed (0.0%), 12,515,679 remaining | 0.85 reports/sec, 2 errors


  Batch 2 reports:   0%|          | 0/1000 [00:00<?, ?it/s]

  Batch 3 reports:   0%|          | 0/1000 [00:00<?, ?it/s]

  💾 Converting 1000 results to Spark DataFrame...
  📊 Counting successes and errors...
     ✓ 995 successful, 5 errors
  ⚠️  Writing 5 errors to default.followup_errors...
     ✓ Errors logged
  🔄 Merging 995 results into default.latest_reports...
     ✓ Merge complete
  📈 Calculating progress...
  📈 Batch 3/12517: 2,990 processed (0.0%), 12,513,687 remaining | 0.90 reports/sec, 5 errors


  Batch 4 reports:   0%|          | 0/1000 [00:00<?, ?it/s]

  💾 Converting 1000 results to Spark DataFrame...
  📊 Counting successes and errors...
     ✓ 995 successful, 5 errors
  ⚠️  Writing 5 errors to default.followup_errors...
     ✓ Errors logged
  🔄 Merging 995 results into default.latest_reports...
     ✓ Merge complete
  📈 Calculating progress...
  📈 Batch 4/12517: 3,985 processed (0.0%), 12,512,692 remaining | 0.85 reports/sec, 5 errors


  Batch 5 reports:   0%|          | 0/1000 [00:00<?, ?it/s]

## Validation & Summary

In [ ]:
# Summary statistics
df_final = spark.table(DEST_TABLE)

print("\n" + "="*60)
print("📊 FOLLOW-UP DETECTION SUMMARY")
print("="*60)

print("\nResults by follow-up status:")
df_final.groupBy("followup_detected").count().orderBy("followup_detected").show()

print("\nResults by follow-up status and confidence:")
df_final.groupBy("followup_detected", "followup_confidence").count().orderBy("followup_detected", "followup_confidence").show()

print("\nProcessing statistics:")
processed = df_final.filter(F.col("followup_processed_at").isNotNull()).count()
print(f"  Total reports: {report_count:,}")
print(f"  Processed: {processed:,} ({100*processed/report_count:.1f}%)")
print(f"  Unprocessed: {report_count - processed:,}")

followup_count = df_final.filter(F.col("followup_detected") == True).count()
if processed > 0:
    print(f"  Follow-up rate: {100*followup_count/processed:.1f}%")

followup_high_conf = df_final.filter((F.col("followup_detected") == True) & (F.col("followup_confidence") == "high")).count()
if followup_count > 0:
    print(f"  High confidence follow-ups: {followup_high_conf:,} ({100*followup_high_conf/followup_count:.1f}%)")

print("\n🔍 Sample High Confidence Follow-up Reports:")
(df_final
    .filter((F.col("followup_detected") == True) & (F.col("followup_confidence") == "high"))
    .select("message_control_id", "followup_confidence", "followup_snippet")
    .limit(10)
    .show(truncate=80)
)

print("\n🔍 Sample Low Confidence Follow-up Reports:")
(df_final
    .filter((F.col("followup_detected") == True) & (F.col("followup_confidence") == "low"))
    .select("message_control_id", "followup_confidence", "followup_snippet")
    .limit(10)
    .show(truncate=80)
)

# Check errors
df_errors = spark.table(ERROR_TABLE)
error_count = df_errors.count()
print(f"\n⚠️  Error Statistics:")
print(f"  Total errors: {error_count:,}")
if error_count > 0:
    print(f"  Error rate: {100*error_count/(processed+error_count):.2f}%")
    print("\n  Top error types:")
    df_errors.groupBy("error").count().orderBy(F.col("count").desc()).limit(5).show(truncate=80)

# Export samples for validation
print("\n💾 Exporting validation samples...")
output_dir = "/home/jovyan/followup_validation"
os.makedirs(output_dir, exist_ok=True)

# High confidence positive samples
high_conf_sample = (df_final
    .filter((F.col("followup_detected") == True) & (F.col("followup_confidence") == "high"))
    .select("message_control_id", "followup_detected", "followup_confidence", "followup_snippet", "report_text")
    .limit(100)
    .toPandas())
high_conf_sample.to_csv(f"{output_dir}/followup_high_confidence_sample.csv", index=False)
print(f"  ✅ Saved: {output_dir}/followup_high_confidence_sample.csv")

# Low confidence positive samples
low_conf_sample = (df_final
    .filter((F.col("followup_detected") == True) & (F.col("followup_confidence") == "low"))
    .select("message_control_id", "followup_detected", "followup_confidence", "followup_snippet", "report_text")
    .limit(100)
    .toPandas())
low_conf_sample.to_csv(f"{output_dir}/followup_low_confidence_sample.csv", index=False)
print(f"  ✅ Saved: {output_dir}/followup_low_confidence_sample.csv")

# Negative samples
negative_sample = (df_final
    .filter(F.col("followup_detected") == False)
    .select("message_control_id", "followup_detected", "followup_confidence", "followup_snippet", "report_text")
    .limit(100)
    .toPandas())
negative_sample.to_csv(f"{output_dir}/followup_negative_sample.csv", index=False)
print(f"  ✅ Saved: {output_dir}/followup_negative_sample.csv")

print("\n" + "="*60)
print("🎉 PROCESSING COMPLETE")
print("="*60)

In [ ]:
# Interactive Dashboard Review
print("\n" + "="*60)
print("📱 INTERACTIVE DASHBOARD REVIEW")
print("="*60)

from followup_review_dashboard import create_review_dashboard

# Stratified sample across modalities AND classifications
df_processed = df_final.filter(F.col("followup_processed_at").isNotNull())

print("\n🔄 Creating stratified sample by modality...")
# Get 10 samples per modality per classification type
df_review = (df_processed
   .filter(F.col("modality").isNotNull())
   .withColumn("category", F.concat_ws("_",
                                       F.col("modality"),
                                       F.col("followup_detected").cast("string"),
                                       F.coalesce(F.col("followup_confidence"), F.lit("null"))))
   .withColumn("row_num", F.row_number().over(
       Window.partitionBy("category").orderBy(F.rand())
   ))
   .filter(F.col("row_num") <= 10)  # 10 per category
   .select("message_control_id", "modality", "followup_detected",
           "followup_confidence", "followup_snippet", "report_text")
   .toPandas())

print(f"\n📊 Loaded {len(df_review)} reports stratified by modality")
print("\nSample distribution:")
print(df_review.groupby(['modality', 'followup_detected', 'followup_confidence']).size())

# Launch dashboard
print("\n🚀 Launching interactive dashboard...")
create_review_dashboard(df_review, report_col='report_text')